##### Copyright 2024 Google LLC.

In [31]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Quickstart with Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/tutorials/python_quickstart"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Google AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This quickstart demonstrates how to use the Python SDK for the Gemini API, which gives you access to Google's Gemini large language models. In this quickstart, you will learn how to:

1. Set up your development environment and API access to use Gemini.
2. Generate text responses from text inputs.
3. Generate text responses from multimodal inputs (text and images).
4. Use Gemini for multi-turn conversations (chat).
5. Use embeddings for large language models.

## Prerequisites

You can run this quickstart in [Google Colab](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb), which runs this notebook directly in the browser and does not require additional environment configuration.

Alternatively, to complete this quickstart locally, ensure that your development environment meets the following requirements:

-  Python 3.9+
-  An installation of `jupyter` to run the notebook.

## Setup

### Install the Python SDK

The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:

In [32]:
!pip install -q -U google-generativeai

### Import packages

Import the necessary packages.

In [33]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [34]:
# Used to securely store your API key
from google.colab import userdata

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [35]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

# the annotation loop

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import time
df=pd.read_csv("/content/drive/MyDrive/data_scrapping/AQMAR/AQMAR_dataset.csv")
# x=840
x= 2277
df=df.iloc[x:].reset_index()
# aqmar


model = genai.GenerativeModel('gemini-pro')

# Apply template function
def apply_template(row):
    template = "**اذكر كلا من معنى و تعريف كلمة ({Word}) في سياق الجملة التالية:**\n\n{Sentence}\n\n**ملاحظة:**\n* تأكد من أنك تشرح معنى الكلمة في سياق الجملة. * حاول استخدام مصادر موثوقة مثل المعاجم أو القواميس. * اكتب إجابتك بشكل واضح ومختصر."
    filled_template = template.format(Word=row['Word'], Sentence=row['Sentence'])
    response1 = model.generate_content(filled_template)
    response2 = model.generate_content(response1.text + "ضع النص التالي في شكل قاموس")

    return {"response1": response1.text, "response2": response2.text}

responses = []

# Process each record with a rate limit of 30 per minute
for index, row in df.iterrows():
    print(f"index= {index} / {df.shape[0]} / {x}")
    # if apply_template(row):
    responses.append(apply_template(row))
    x+=1
    # else:
    #   print('none')

    time.sleep(2)  # Sleep for 2 seconds to maintain the rate of 30 records/min

    # Save a checkpoint every record (for demonstration; typically would be every 60 records)
    if (index + 1) % 60 == 0:
        checkpoint_filename = f'/content/drive/MyDrive/data_scrapping/AQMAR/checkpoint_{index + 1}.csv'
        checkpoint_df = df.iloc[:index + 1].join(pd.DataFrame(responses))

        checkpoint_df.to_csv(checkpoint_filename, index=False)
        print(f'Checkpoint saved: {checkpoint_filename}')



# Convert the list of dictionaries to a DataFrame and join with the original DataFrame
response_df = pd.DataFrame(responses)
df = df.join(response_df)
df
# Display the DataFrame
print(df.head())

# Save the final DataFrame
final_filename = '/content/drive/MyDrive/data_scrapping/AQMAR/final_OUTPUT.csv'
df.to_csv(final_filename, index=False)
print(f'Final output saved: {final_filename}')


index= 0 / 65933 / 2277
index= 1 / 65933 / 2278
index= 2 / 65933 / 2279
index= 3 / 65933 / 2280
index= 4 / 65933 / 2281
index= 5 / 65933 / 2282
index= 6 / 65933 / 2283
index= 7 / 65933 / 2284
index= 8 / 65933 / 2285
index= 9 / 65933 / 2286
index= 10 / 65933 / 2287
index= 11 / 65933 / 2288
